# Model Selection for SLP Prediction

This notebook performs model selection to predict the `slp` column using various machine learning algorithms with time series cross-validation.


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.base import clone
from lightgbm import LGBMRegressor
from joblib import Parallel, delayed
import joblib

import itertools
import warnings
warnings.filterwarnings('ignore')


## 1. Load and Prepare Data


In [2]:
# Load the dataset
data_name = 'stat'
out_path = f'results/{data_name}'
dataset = f'dataset/data_v3_{data_name}.csv'
df = pd.read_csv(dataset, sep=';', decimal=',')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

Dataset shape: (1095, 13)

Columns: ['date', 'slp', 'temperature_2m_mean', 'sunrise', 'et0_fao_evapotranspiration', 'sunshine_duration', 'snowfall_sum', 'day_of_year', 'precipitation_hours', 'weathercode', 'windspeed_10m_max', 'rain_sum', 'holiday']


,date,slp,temperature_2m_mean,sunrise,et0_fao_evapotranspiration,sunshine_duration,snowfall_sum,day_of_year,precipitation_hours,weathercode,windspeed_10m_max,rain_sum,holiday
0,2022-10-01,693745.937,0.030121,0.132183,-0.551434,-0.944343,-0.160809,274,0.480580,61,0.697061,0.969397,0
1,2022-10-02,633340.108,0.204324,0.143013,-0.204844,0.522740,-0.160809,275,0.065976,55,1.031850,0.126311,0
2,2022-10-03,621820.506,0.217724,0.164674,-0.359572,-0.105130,-0.160809,276,-0.763233,3,1.398524,-0.475894,1
3,2022-10-04,549282.563,0.271326,0.186334,-0.396706,0.014750,-0.160809,277,-0.763233,3,-0.514557,-0.475894,0
4,2022-10-05,483362.562,0.485730,0.207995,0.024153,0.337364,-0.160809,278,-0.763233,3,0.314445,-0.475894,0


In [3]:
# Parse date and sort by date (important for time series)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

In [4]:
# Separate features and target

X = df.drop(columns=['date', 'slp'])
y = df['slp']

# Define feature types for proper preprocessing
boolean_cols = ['holiday']
categorical_cols = ['weathercode']  # Leave as-is for tree-based models
cyclical_cols = ['day_of_week', 'day_of_year', 'winddirection_10m_dominant']

# All other columns are continuous and should be scaled
continuous_cols = [col for col in X.columns 
                   if col not in boolean_cols + categorical_cols + cyclical_cols]

test_days = 365
X_test = X.iloc[-test_days:]
y_test = y.iloc[-test_days:]

X_train = X.iloc[:-test_days]
y_train = y.iloc[:-test_days]


## 3. Time Series Split

In [5]:
n_split = 10
tscv = TimeSeriesSplit(n_splits = n_split)

In [6]:
def evaluate_model_params(estimator, X, y, tscv):
    """Valuta un singolo estimator (clonato) con TimeSeriesSplit.
    Ritorna dizionario di metriche medie.
    """
    rmse_scores = []
    mae_scores = []
    r2_scores = []
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        model = clone(estimator)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse_scores.append(np.sqrt(mean_squared_error(y_test, y_pred)))
        mae_scores.append(mean_absolute_error(y_test, y_pred))
        r2_scores.append(r2_score(y_test, y_pred))
    return {
        'RMSE_mean': np.mean(rmse_scores),
        'RMSE_std': np.std(rmse_scores),
        'MAE_mean': np.mean(mae_scores),
        'MAE_std': np.std(mae_scores),
        'R2_mean': np.mean(r2_scores),
        'R2_std': np.std(r2_scores),
    }

In [7]:
def param_grid_iter(grid_dict):
    """Genera tutte le combinazioni dalla dict di liste come sklearn.model_selection.ParameterGrid.
    grid_dict: {'param': [v1,v2,...], ...}
    """
    keys = list(grid_dict.keys())
    for values in itertools.product(*(grid_dict[k] for k in keys)):
        yield dict(zip(keys, values))

In [8]:
def top_k_results(results_list, k=5, metric='R2_mean'):
    """Ordina la lista di dict (ognuno con 'params' e metriche) e ritorna top k.
    metric: campo su cui ordinare (default R2_mean decrescente).
    """
    return sorted(results_list, key=lambda r: r.get(metric, -np.inf), reverse=True)[:k]


In [9]:
def make_fine_grid_around(best_params, param_specs, factor=0.5, n_points=5):
    """Crea una lista di param dict per la fine search intorno ai best_params.
    param_specs for each param: {'type':'int'/'float'/'cat', 'bounds':(min,max)}
    factor: estensione percentuale (es 0.5 = +/-50%)
    n_points: quanti punti generare per ogni parametro
    """
    fine_specs = {}
    for p, spec in param_specs.items():
        best = best_params.get(p, None)
        if best is None:
            # se non presente, usa bounds
            lo, hi = spec.get('bounds', (None, None))
            if spec['type'] == 'cat':
                fine_specs[p] = spec['values']
            elif spec['type'] == 'int':
                fine_specs[p] = list(range(
                    max(1, int(lo)),
                    int(hi) + 1,
                    max(1, int((int(hi)-int(lo))//(n_points-1) if n_points>1 else 1))
                ))
            else:
                fine_specs[p] = list(np.linspace(lo, hi, n_points))
            continue

        if spec['type'] == 'cat':
            fine_specs[p] = spec['values']

        elif spec['type'] == 'int':
            lo = max(spec['bounds'][0], int(best - max(1, factor * best)))
            hi = min(spec['bounds'][1], int(best + max(1, factor * best)))
            if lo >= hi:
                fine_specs[p] = [int(best)]
            else:
                fine_specs[p] = sorted(list(set([int(x) for x in np.linspace(lo, hi, n_points)])))

        else:  # float
            lo = max(spec['bounds'][0], best * (1 - factor))
            hi = min(spec['bounds'][1], best * (1 + factor))
            fine_specs[p] = list(np.linspace(lo, hi, n_points))

    combos = list(itertools.islice(param_grid_iter(fine_specs), 10000))
    return combos


# Define Models

In [10]:
models_space = {
    'RandomForest': {
        'estimator': RandomForestRegressor(random_state=42, n_jobs=-1),
        'coarse': {
            'n_estimators': [50, 100, 300],
            'max_depth': [5, 10, 20, None],
            'min_samples_split': [2, 5, 10],
            'max_features': ['sqrt', 'log2', 0.5]
        },
        'specs': {
            'n_estimators': {'type':'int', 'bounds':(10,1000)},
            'max_depth': {'type':'int', 'bounds':(3,50)},
            'min_samples_split': {'type':'int', 'bounds':(2,50)},
            'max_features': {'type':'cat', 'values':['sqrt','log2',0.2,0.3,0.4,0.5,None]}
        }
    },
    'GradientBoosting': {
        'estimator': GradientBoostingRegressor(random_state=42),
        'coarse': {
            'n_estimators': [100, 300, 800],
            'learning_rate': [0.01, 0.05, 0.1],
            'max_depth': [3, 5, 8],
            'subsample': [0.6, 0.8, 1.0]
        },
        'specs': {
            'n_estimators': {'type':'int', 'bounds':(50,2000)},
            'learning_rate': {'type':'float', 'bounds':(1e-4,1.0)},
            'max_depth': {'type':'int', 'bounds':(1,20)},
            'subsample': {'type':'float', 'bounds':(0.3,1.0)}
        }
    },
    # 'LightGBM': {
    #     'estimator': LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1),
    #     'coarse': {
    #         'n_estimators': [100, 300, 1000],
    #         'learning_rate': [0.01, 0.05, 0.1],
    #         'num_leaves': [15, 31, 63],
    #         'max_depth': [-1, 5, 10]
    #     },
    #     'specs': {
    #         'n_estimators': {'type':'int', 'bounds':(50,3000)},
    #         'learning_rate': {'type':'float', 'bounds':(1e-4,1.0)},
    #         'num_leaves': {'type':'int', 'bounds':(6,2048)},
    #         'max_depth': {'type':'int', 'bounds':(-1,50)},
    #         'min_child_samples': {'type':'int', 'bounds':(1,100)}
    #     }
    # }
}

In [11]:
def evaluate_param_set(estimator, params, X, y, tscv):
    est = clone(estimator).set_params(**params)
    metrics = evaluate_model_params(est, X, y, tscv)
    return {'params': params, **metrics}

In [12]:
def coarse_to_fine_search(name, model_info, X, y, tscv, top_k=5):
    print('\n' + '='*60)
    print(f'Inizio ricerca per: {name}')
    estimator = model_info['estimator']
    coarse_grid = model_info['coarse']
    specs = model_info['specs']

    # ------- COARSE SEARCH PARALLEL -------
    param_list = list(param_grid_iter(coarse_grid))
    print(f'Coarse grid size: {len(param_list)} combinazioni')

    coarse_results = Parallel(n_jobs=-1, verbose=10)(
        delayed(evaluate_param_set)(estimator, params, X, y, tscv)
        for params in param_list
    )

    # top-k
    top_coarse = top_k_results(coarse_results, k=top_k, metric='R2_mean')
    print('\nTop risultati (coarse):')
    for r in top_coarse:
        print(f"  R2={r['R2_mean']:.4f} — params={r['params']}")

    # ------- FINE SEARCH PARALLEL -------
    best_coarse = top_coarse[0]
    best_params = best_coarse['params']

    fine_param_list = make_fine_grid_around(best_params, specs, factor=0.5, n_points=7)
    print(f"\nFine grid size (limitata): {len(fine_param_list)} combinazioni\n")

    fine_results = Parallel(n_jobs=-1, verbose=10)(
        delayed(evaluate_param_set)(estimator, params, X, y, tscv)
        for params in fine_param_list
    )

    top_fine = top_k_results(fine_results, k=3, metric='R2_mean')
    print('\nTop risultati (fine):')
    for r in top_fine:
        print(f"  R2={r['R2_mean']:.4f} — params={r['params']}")

    best_final = top_fine[0]
    return {
        'coarse_results': coarse_results,
        'top_coarse': top_coarse,
        'fine_results': fine_results,
        'top_fine': top_fine,
        'best': best_final
    }


In [13]:
all_best = {}
for name, info in models_space.items():
    res = coarse_to_fine_search(name, info, X_train, y_train, tscv, top_k=3)
    best_entry = res['best']
    all_best[name] = best_entry
    # salva i risultati intermedi su disco per controllo laterale
    joblib.dump(res, f'{out_path}/results_{name}_coarse_to_fine_{data_name}.pkl')
    print(f"Risultati salvati in {out_path}/results_{name}_coarse_to_fine_{data_name}.pkl")


Inizio ricerca per: RandomForest
Coarse grid size: 108 combinazioni


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 112 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of 108 | elapsed:    5.4s remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  17 out of 108 | elapsed:    5.6s remaining:   29.8s
[Parallel(n_jobs=-1)]: Done  28 out of 108 | elapsed:    5.7s remaining:   16.3s
[Parallel(n_jobs=-1)]: Done  39 out of 108 | elapsed:    7.0s remaining:   12.3s
[Parallel(n_jobs=-1)]: Done  50 out of 108 | elapsed:    7.1s remaining:    8.2s
[Parallel(n_jobs=-1)]: Done  61 out of 108 | elapsed:    7.2s remaining:    5.5s
[Parallel(n_jobs=-1)]: Done  72 out of 108 | elapsed:    7.4s remaining:    3.7s
[Parallel(n_jobs=-1)]: Done  83 out of 108 | elapsed:   10.5s remaining:    3.2s
[Parallel(n_jobs=-1)]: Done  94 out of 108 | elapsed:   10.5s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done 105 out of 108 | elapsed:   10.9s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   11.2s finished
[Parallel(n_jobs=-1)]: Using backend Loky


Top risultati (coarse):
  R2=-6.1651 — params={'n_estimators': 50, 'max_depth': None, 'min_samples_split': 2, 'max_features': 0.5}
  R2=-6.1668 — params={'n_estimators': 50, 'max_depth': 20, 'min_samples_split': 2, 'max_features': 0.5}
  R2=-6.2868 — params={'n_estimators': 300, 'max_depth': None, 'min_samples_split': 2, 'max_features': 0.5}

Fine grid size (limitata): 686 combinazioni



[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 226 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 257 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 288 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 321 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 389 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 424 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done 461 tasks      | elapsed:   14.3s
[Paralle


Top risultati (fine):
  R2=-5.9841 — params={'n_estimators': 75, 'max_depth': 10, 'min_samples_split': 3, 'max_features': None}
  R2=-6.0380 — params={'n_estimators': 58, 'max_depth': 10, 'min_samples_split': 2, 'max_features': None}
  R2=-6.0869 — params={'n_estimators': 41, 'max_depth': 10, 'min_samples_split': 2, 'max_features': None}
Risultati salvati in results/stat/results_RandomForest_coarse_to_fine_stat.pkl

Inizio ricerca per: GradientBoosting
Coarse grid size: 81 combinazioni


[Parallel(n_jobs=-1)]: Done   2 out of  81 | elapsed:    1.4s remaining:   56.9s
[Parallel(n_jobs=-1)]: Done  11 out of  81 | elapsed:    2.1s remaining:   13.1s
[Parallel(n_jobs=-1)]: Done  20 out of  81 | elapsed:    2.8s remaining:    8.6s
[Parallel(n_jobs=-1)]: Done  29 out of  81 | elapsed:    4.2s remaining:    7.5s
[Parallel(n_jobs=-1)]: Done  38 out of  81 | elapsed:    5.5s remaining:    6.2s
[Parallel(n_jobs=-1)]: Done  47 out of  81 | elapsed:    6.7s remaining:    4.8s
[Parallel(n_jobs=-1)]: Done  56 out of  81 | elapsed:    9.8s remaining:    4.4s
[Parallel(n_jobs=-1)]: Done  65 out of  81 | elapsed:   12.0s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  74 out of  81 | elapsed:   15.0s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:   19.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 112 concurrent workers.



Top risultati (coarse):
  R2=-4.0370 — params={'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 8, 'subsample': 1.0}
  R2=-4.0370 — params={'n_estimators': 800, 'learning_rate': 0.1, 'max_depth': 8, 'subsample': 1.0}
  R2=-4.0382 — params={'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 8, 'subsample': 1.0}

Fine grid size (limitata): 2401 combinazioni



[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 197 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 226 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 257 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 288 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 321 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done 389 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done 424 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done 461 tasks      | elapsed:   26.8s
[Paralle


Top risultati (fine):
  R2=-3.6828 — params={'n_estimators': 350, 'learning_rate': np.float64(0.15000000000000002), 'max_depth': 6, 'subsample': np.float64(0.5)}
  R2=-3.6829 — params={'n_estimators': 450, 'learning_rate': np.float64(0.15000000000000002), 'max_depth': 6, 'subsample': np.float64(0.5)}
  R2=-3.6830 — params={'n_estimators': 400, 'learning_rate': np.float64(0.15000000000000002), 'max_depth': 6, 'subsample': np.float64(0.5)}
Risultati salvati in results/stat/results_GradientBoosting_coarse_to_fine_stat.pkl


In [14]:
summary = []
for name, best in all_best.items():
    summary.append({
        'model': name,
        'R2_mean': best['R2_mean'],
        'RMSE_mean': best['RMSE_mean'],
        'MAE_mean': best['MAE_mean'],
        'best_params': best['params']
    })
summary_df = pd.DataFrame(summary).sort_values('R2_mean', ascending=False).reset_index(drop=True)
print('\n' + '='*80)
print('CONFRONTO FINALE: modelli ottimizzati')
print(summary_df)


CONFRONTO FINALE: modelli ottimizzati
              model   R2_mean      RMSE_mean       MAE_mean  \
0  GradientBoosting -3.682819  200385.396284  164186.833704   
1      RandomForest -5.984121  208557.535249  171783.291551   

                                         best_params  
0  {'n_estimators': 350, 'learning_rate': 0.15000...  
1  {'n_estimators': 75, 'max_depth': 10, 'min_sam...  


In [15]:
print('\nFit e salvataggio dei modelli finali (su tutto il dataset):')
for idx, row in summary_df.iterrows():
    name = row['model']
    best_params = row['best_params']
    estimator = models_space[name]['estimator'].set_params(**best_params)
    print(f"  Fit model: {name} con params: {best_params}")
    estimator.fit(X_train, y_train)
    joblib.dump(estimator, f'{out_path}/best_model_{name}_{data_name}.pkl')
    print(f"  Salvato: {out_path}/best_model_{name}_{data_name}.pkl")

print('\nDONE')


Fit e salvataggio dei modelli finali (su tutto il dataset):
  Fit model: GradientBoosting con params: {'n_estimators': 350, 'learning_rate': np.float64(0.15000000000000002), 'max_depth': 6, 'subsample': np.float64(0.5)}


  Salvato: results/stat/best_model_GradientBoosting_stat.pkl
  Fit model: RandomForest con params: {'n_estimators': 75, 'max_depth': 10, 'min_samples_split': 3, 'max_features': None}
  Salvato: results/stat/best_model_RandomForest_stat.pkl

DONE


In [16]:
def evaluate_on_test(estimator, X_test, y_test):
    """Valuta un modello già fit su un test set con metriche aggiuntive."""
    y_pred = estimator.predict(X_test)
    
    bias = np.mean(y_pred - y_test)
    max_error = np.max(np.abs(y_pred - y_test))
    pearson_corr = pearsonr(y_test, y_pred)[0]
    spearman_corr = spearmanr(y_test, y_pred)[0]
    mape = np.mean(np.abs((y_test - y_pred)/y_test)) * 100  # attenzione valori vicino a zero
    smape = np.mean(np.abs(y_test - y_pred)/((np.abs(y_test)+np.abs(y_pred))/2)) * 100
    
    return {
        'R2': r2_score(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAE': mean_absolute_error(y_test, y_pred),
        'Bias': bias,
        'Max_Error': max_error,
        'Pearson': pearson_corr,
        'Spearman': spearman_corr,
        'MAPE': mape,
        'sMAPE': smape
    }


In [17]:
test_results = []

for idx, row in summary_df.iterrows():
    name = row['model']
    model_file = f'{out_path}/best_model_{name}_{data_name}.pkl'
    
    # Carica modello già fit
    model = joblib.load(model_file)
    
    # Valutazione sul test set
    metrics = evaluate_on_test(model, X_test, y_test)
    
    metrics['model'] = name
    metrics['best_params'] = row['best_params']
    test_results.append(metrics)

# Trasforma in DataFrame
test_results_df = pd.DataFrame(test_results).sort_values('R2', ascending=False).reset_index(drop=True)

# Stampa
print('\n=== Risultati sui test set ===')
print(test_results_df)



=== Risultati sui test set ===
         R2           RMSE           MAE          Bias     Max_Error  \
0  0.977886  102342.387311  67609.947432 -19923.536554  494618.51268   
1  0.974090  110779.547178  73721.649296 -20068.810195  492312.89677   

    Pearson  Spearman      MAPE     sMAPE             model  \
0  0.989354  0.988690  8.124447  8.176765      RandomForest   
1  0.987412  0.986331  8.759123  8.969314  GradientBoosting   

                                         best_params  
0  {'n_estimators': 75, 'max_depth': 10, 'min_sam...  
1  {'n_estimators': 350, 'learning_rate': 0.15000...  


In [18]:
# Salva in CSV
test_results_df.to_csv(f'{out_path}/test_set_results_{data_name}.csv', index=False)

# Salva anche in pickle per uso successivo
joblib.dump(test_results_df, f'{out_path}/test_set_results_{data_name}.pkl')

print(f"\nRisultati test set salvati in '{out_path}/test_set_results_{data_name}.csv' e '{out_path}/test_set_results_{data_name}.pkl'")


Risultati test set salvati in 'results/stat/test_set_results_stat.csv' e 'results/stat/test_set_results_stat.pkl'
